# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys
import datetime
import numpy as np
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, \
                                            Add, LSTM

from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.metrics import classification_report

from tqdm.auto import tqdm

In [3]:
os.chdir("/content/drive/MyDrive/Projects/AlarmWaterClassification")

# Utilies

In [4]:
def print_M(conf_M, class_names):
    s = "activity," + ",".join(class_names)
    print(s)
    for i, row in enumerate(conf_M):
        print(class_names[i] + "," + ",".join(map(str, row)))

def print_M_P(conf_M, class_names):
    s = "activity," + ",".join(class_names)
    print(s)
    for i, row in enumerate(conf_M):
        total = sum(row)
        percentages = [str(round(val / total, 2)) if total > 0 else '0' for val in row]
        print(class_names[i] + "," + ",".join(percentages))

def showResult(result, y_test, class_names):
    predictions = [np.argmax(y) for y in result]
    expected = [np.argmax(y) for y in y_test]

    num_labels = y_test.shape[1]
    conf_M = [[0] * num_labels for _ in range(num_labels)]

    for e, p in zip(expected, predictions):
        conf_M[e][p] += 1

    print_M(conf_M, class_names)
    print_M_P(conf_M, class_names)

def load_weight(path):
    model = load_model(path)
    print(model.summary())
    return model

# Model

In [58]:
def build_improved_model(input_shape, num_labels):

    model = Sequential()

    model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.5))

    model.add(LSTM(64))
    model.add(Dropout(0.5))

    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_labels))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

# Data

## Data loading

In [64]:
model_weight_out = os.path.join('weights', 'exp_model_16k_1.3.weights.h5')

# if os.path.exists(model_weight_out):
#     sys.exit(f"The same file name exists already: {model_weight_out}")

############### Loading the datasets #####################

print('\nLoading the data\n')

featuresPath = "features_16k/"

class_names = np.load(os.path.join(featuresPath, 'class_names.npy'))

X_train = np.load(os.path.join(featuresPath, 'X_train.npy'))
y_train = np.load(os.path.join(featuresPath, 'y_train.npy'))

X_val = np.load(os.path.join(featuresPath, 'X_val.npy'))
y_val = np.load(os.path.join(featuresPath, 'y_val.npy'))

X_test = np.load(os.path.join(featuresPath, 'X_test.npy'))
y_test = np.load(os.path.join(featuresPath, 'y_test.npy'))

num_labels = y_train.shape[1]

## Data Balancing

In [60]:
print("\nBalancing the data\n")

print("Train Class distribution before balancing:", Counter(np.argmax(y_train, axis=1)))

# Upsampling using SMOTE
smote = SMOTE(sampling_strategy={1: 10000, 2: 9000})
oversampled_features, oversampled_labels = smote.fit_resample(X_train, y_train)

# Downsampling using RandomUnderSampler
undersampler = RandomUnderSampler(sampling_strategy={0: 7300})
undersampled_features, undersampled_labels = undersampler.fit_resample(
    oversampled_features, oversampled_labels)

print("Train Class distribution after balancing:", Counter(
    np.argmax(undersampled_labels, axis=1)))

X_train = undersampled_features
y_train = undersampled_labels


Balancing the data

Train Class distribution before balancing: Counter({0: 7343, 1: 4309, 2: 842})


/usr/local/lib/python3.10/dist-packages/imblearn/utils/_validation.py:313: UserWarning: After over-sampling, the number of samples (10000) in class 1 will be larger than the number of samples in the majority class (class #0 -> 7343)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/imblearn/utils/_validation.py:313: UserWarning: After over-sampling, the number of samples (9000) in class 2 will be larger than the number of samples in the majority class (class #0 -> 7343)
  warnings.warn(


Train Class distribution after balancing: Counter({1: 10000, 2: 9000, 0: 7300})


# Training

In [61]:
X_train = np.expand_dims(X_train, axis=1)
X_val = np.expand_dims(X_val, axis=1)
X_test = np.expand_dims(X_test, axis=1)

In [62]:
###################### Training the model ###########################3
print("\nTraining the model\n")

input_shape = (X_train.shape[1], X_train.shape[2])
model = build_improved_model(input_shape, num_labels)
# model.summary()

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))

callback = LearningRateScheduler(scheduler)

model.fit(X_train, y_train, batch_size=32, epochs=20,
          validation_data=(X_val, y_val),
          callbacks=[callback])


Training the model

Epoch 1/20
822/822 [==============================] - 11s 8ms/step - loss: 0.6552 - accuracy: 0.7304 - val_loss: 0.8135 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 2/20
822/822 [==============================] - 7s 8ms/step - loss: 0.4126 - accuracy: 0.8410 - val_loss: 0.9761 - val_accuracy: 0.6263 - lr: 0.0010
Epoch 3/20
822/822 [==============================] - 6s 7ms/step - loss: 0.3523 - accuracy: 0.8662 - val_loss: 1.0874 - val_accuracy: 0.6314 - lr: 0.0010
Epoch 4/20
822/822 [==============================] - 7s 8ms/step - loss: 0.3242 - accuracy: 0.8798 - val_loss: 1.0600 - val_accuracy: 0.6141 - lr: 0.0010
Epoch 5/20
822/822 [==============================] - 6s 7ms/step - loss: 0.2873 - accuracy: 0.8943 - val_loss: 1.0616 - val_accuracy: 0.6098 - lr: 0.0010
Epoch 6/20
822/822 [==============================] - 7s 9ms/step - loss: 0.2635 - accuracy: 0.9034 - val_loss: 1.0604 - val_accuracy: 0.6350 - lr: 0.0010
Epoch 7/20
822/822 [============================

# Testing

In [63]:
print("\nTesting the model\n")

result = model.predict(X_test)

cnt, cnt_alarm, cnt_other, cnt_water = 0, 0, 0, 0
alarm_num, other_num, water_num = (sum(np.argmax(y_test, axis=1) == 0),
                                    sum(np.argmax(y_test, axis=1) == 1),
                                    sum(np.argmax(y_test, axis=1) == 2))

for i in range(len(y_test)):
    pred = np.argmax(result[i])
    if np.argmax(y_test[i]) == pred:
        cnt += 1
        if pred == 0:
            cnt_alarm += 1
        elif pred == 1:
            cnt_other += 1
        else:
            cnt_water += 1

acc = round(cnt * 100 / len(y_test), 2)
acc_alarm = round(cnt_alarm * 100 / alarm_num, 2)
acc_other = round(cnt_other * 100 / other_num, 2)
acc_water = round(cnt_water * 100 / water_num, 2)

print(
    f"Total Accuracy: {acc}%, Alarm Accuracy: {acc_alarm}%, Others Accuracy: {acc_other}%, Water Accuracy: {acc_water}%")

showResult(result, y_test, class_names)

print("\n")
print(classification_report(
    np.argmax(y_test, axis=1),
    np.argmax(result, axis=1),
    target_names=list(class_names)
))


Testing the model

23/23 [==============================] - 1s 3ms/step
Total Accuracy: 82.08%, Alarm Accuracy: 88.97%, Others Accuracy: 70.97%, Water Accuracy: 78.26%
activity,Alarm,Water,Other
Alarm,379,44,3
Water,58,176,14
Other,4,6,36
activity,Alarm,Water,Other
Alarm,0.89,0.1,0.01
Water,0.23,0.71,0.06
Other,0.09,0.13,0.78


              precision    recall  f1-score   support

       Alarm       0.86      0.89      0.87       426
       Water       0.78      0.71      0.74       248
       Other       0.68      0.78      0.73        46

    accuracy                           0.82       720
   macro avg       0.77      0.79      0.78       720
weighted avg       0.82      0.82      0.82       720



# Saving

In [65]:
if not os.path.exists("Models"):
    os.makedirs("Models")
path = os.path.join("Models", f"audio_NN_New{datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}_acc_{acc}")
model_json = model.to_json()
with open(f"{path}.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(f"{path}.weights.h5")

if not os.path.exists("weights"):
    os.makedirs("weights")
model.save(model_weight_out, overwrite=True, include_optimizer=False)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [66]:
loaded_model = tf.keras.models.load_model(model_weight_out)